# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
#from api_keys import g_key
g_key = os.environ.get('g_key')


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# output of WeatherPy is the input.
city_data = pd.read_csv("../output_data/city_data.csv")
city_df = pd.DataFrame(city_data)
city_df.head()

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jamestown,42.10,-79.24,66.20,88,1,3.36,US,1592659411
1,Cap Malheureux,-19.98,57.61,73.00,70,41,10.00,MU,1592659707
2,Kapaa,22.08,-159.32,77.00,78,20,14.99,US,1592659707
3,Bredasdorp,-34.53,20.04,69.80,60,3,3.60,ZA,1592659707
4,Cidreira,-30.18,-50.21,68.99,89,93,6.17,BR,1592659707


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations and humidity for weight
locations = city_df[["Latitude", "Longitude"]].astype(float)
humid = city_df['Humidity'].astype(float)

In [5]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

* Narrow down the DataFrame to find your ideal weather condition. For example:

  * A max temperature lower than 80 degrees but higher than 70.

  * Wind speed less than 10 mph.

  * Zero cloudiness.

In [6]:
perf_vac = city_df.loc[city_df["Max Temp"] > 70,:]
perf_vac = perf_vac.loc[perf_vac["Max Temp"] < 80,:]
perf_vac = perf_vac.loc[perf_vac['Wind Speed'] < 10,:]
perf_vac = perf_vac.loc[perf_vac['Cloudiness'] == 0,:]

perf_vac_df = pd.DataFrame(perf_vac)
perf_vac_df
# there are no null values in the dataframe to drop

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
157,Roebourne,-20.78,117.13,71.91,55,0,3.85,AU,1592659730
202,Cherskiy,68.75,161.30,71.26,36,0,5.06,RU,1592659736
211,Lüderitz,-26.65,15.16,78.80,22,0,8.05,NaN,1592659737
247,Prainha,-1.80,-53.48,78.58,89,0,4.21,BR,1592659742
274,Emilio Carranza,19.97,-96.61,78.80,72,0,4.25,MX,1592659746
343,Belaya Gora,68.53,146.42,75.51,32,0,5.84,RU,1592659758
382,Oranjemund,-28.55,16.43,70.83,38,0,8.93,NaN,1592659564
385,Beausoleil,43.75,7.42,75.00,73,0,9.17,FR,1592659763
390,Zhigansk,66.77,123.37,70.16,43,0,4.85,RU,1592659764
501,Kochkurovo,54.04,45.42,76.24,63,0,5.39,RU,1592659779


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perf_vac_df
hotel_df['Hotel Name'] = np.nan

target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Latitude']}, {row['Longitude']}",
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    #response_json = response.json()
    results = response['results']
    if len(results) > 0:
        name = response['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

hotel_df

,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
157,Roebourne,-20.78,117.13,71.91,55,0,3.85,AU,1592659730,Latitude20 Roebourne Village
202,Cherskiy,68.75,161.30,71.26,36,0,5.06,RU,1592659736,NaN
211,Lüderitz,-26.65,15.16,78.80,22,0,8.05,NaN,1592659737,LÜDERITZ NEST HOTEL
247,Prainha,-1.80,-53.48,78.58,89,0,4.21,BR,1592659742,palace hotel
274,Emilio Carranza,19.97,-96.61,78.80,72,0,4.25,MX,1592659746,Hotel Carranza Inn
343,Belaya Gora,68.53,146.42,75.51,32,0,5.84,RU,1592659758,NaN
382,Oranjemund,-28.55,16.43,70.83,38,0,8.93,NaN,1592659564,NaN
385,Beausoleil,43.75,7.42,75.00,73,0,9.17,FR,1592659763,Hotel de Paris Monte-Carlo
390,Zhigansk,66.77,123.37,70.16,43,0,4.85,RU,1592659764,Gostinichnyy Kompleks Aylgy
501,Kochkurovo,54.04,45.42,76.24,63,0,5.39,RU,1592659779,NaN


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))